# Retrieval Augmented Generation - RAG 

We will use the RAG technique to use language models to attempt to solve the multi-class, multi-label classification problem.

In [ ]:
from utils import parse_sgm_to_dataframe

## Data Exploration

In [ ]:
df = parse_sgm_to_dataframe('../../data/reuters21578/reut2-000.sgm')

## Find suitable pre-trained models

## Explore the use of models pretrained on article data

## Optimize our RAG pipeline

## Explore retrieval improvements

Retrieval is in-expensive so it is in most cases a good tradeoff to do more on the retrieval side in order to ensure that we have a good list of candidate labels that includes the true labels.

To explore:
- Sentence splitting
- Retrieval ensemble